In [1]:
import pandas as pd

In [3]:
FAMILY = "family"
CIRCUMSTANCE = "circ-"

columns = ["V" + str(x) for x in range(1, 153)]
death_eligible_circumstances = { 3, 5, 2, 9, 46, 49, 48 }

files = ["shr/shr_2015/shr_2015.xlsx", "shr/shr_2014/shr_2014.xlsx", "shr/shr_2013/shr_2013.xlsx", "shr/shr_2012/shr_2012.xlsx", "shr/shr_2011/shr_2011.xlsx", "shr/shr_2010/shr_2010.xlsx", "shr/shr_2009/shr_2009.xlsx", "shr/shr_2008/shr_2008.xlsx", "shr/shr_2007/shr_2007.xlsx", "shr/shr_2006/shr_2006.xlsx"]

df = pd.DataFrame()
for f in files:
    print(f)
    df_part = pd.read_excel(f)
    df_part.columns = columns
    df = df.append(df_part, ignore_index=True)

shr/shr_2015/shr_2015.xlsx
shr/shr_2014/shr_2014.xlsx
shr/shr_2013/shr_2013.xlsx
shr/shr_2012/shr_2012.xlsx
shr/shr_2011/shr_2011.xlsx
shr/shr_2010/shr_2010.xlsx
shr/shr_2009/shr_2009.xlsx
shr/shr_2008/shr_2008.xlsx
shr/shr_2007/shr_2007.xlsx
shr/shr_2006/shr_2006.xlsx


In [4]:
print(list(df.columns))
df.head()

['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V29', 'V30', 'V31', 'V32', 'V33', 'V34', 'V35', 'V36', 'V37', 'V38', 'V39', 'V40', 'V41', 'V42', 'V43', 'V44', 'V45', 'V46', 'V47', 'V48', 'V49', 'V50', 'V51', 'V52', 'V53', 'V54', 'V55', 'V56', 'V57', 'V58', 'V59', 'V60', 'V61', 'V62', 'V63', 'V64', 'V65', 'V66', 'V67', 'V68', 'V69', 'V70', 'V71', 'V72', 'V73', 'V74', 'V75', 'V76', 'V77', 'V78', 'V79', 'V80', 'V81', 'V82', 'V83', 'V84', 'V85', 'V86', 'V87', 'V88', 'V89', 'V90', 'V91', 'V92', 'V93', 'V94', 'V95', 'V96', 'V97', 'V98', 'V99', 'V100', 'V101', 'V102', 'V103', 'V104', 'V105', 'V106', 'V107', 'V108', 'V109', 'V110', 'V111', 'V112', 'V113', 'V114', 'V115', 'V116', 'V117', 'V118', 'V119', 'V120', 'V121', 'V122', 'V123', 'V124', 'V125', 'V126', 'V127', 'V128', 'V129', 'V130', 'V131', 'V132', 'V133', 'V134', 'V135', 'V136', 'V137', 'V138', 'V1

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V143,V144,V145,V146,V147,V148,V149,V150,V151,V152
0,6,1,AL00112,3,6,2015,85163,37,98,0,...,,,,,,,,,,
1,6,1,AL00112,3,6,2015,85163,37,98,0,...,,,,,,,,,,
2,6,1,AL00112,3,6,2015,85163,37,98,0,...,,,,,,,,,,
3,6,2,AZ00189,7,8,2015,0,,,0,...,,,,,,,,,,
4,6,2,AZ00189,7,8,2015,0,,,0,...,,,,,,,,,,


In [70]:
death_eligible_circumstances = { 3, 5, 2, 9, 46, 49, 48 }
fam_cases = {"BF", "BR", "CH", "CW", "DA", "FA", "GF", "HO", "HU", "MO", "SD", "SF", "SI", "MO", "SO", "SS", "WI" }
circumstance_columns = ["V29","V79","V87","V95","V103","V111","V119","V127","V135","V143","V151"]
fam_columns = ["V28","V78","V86","V94","V102","V110","V118","V126","V134","V142","V150"]
age_columns = ["V19", "V33", "V37", "V41", "V45", "V49", "V53", "V57", "V61", "V65", "V69"]
offender_age_columns = ["V23","V73","V81","V89","V97","V105","V113","V121","V129","V137","V145"]



c = 0


def label_unique(row):
    return '' + row['V3'] + '-' + str(row['V17'])

def label_death_eligible(row):    
    num_18 = count_under_18(row)
    
    counts = createDeathMap();
    
    if not check_offender_age(row): #stops if all perps are less than 18
        return counts
    
    if row['V31'] > 0 or check_age(row): #multiple victims or very young victim
        counts["MULT_VICTIMS"] += 1 + row['V32'] - num_18
        return counts;
    return check_circumstance(row, counts) #based on circumstance of each perp

def createDeathMap():
    maps = { circ : 0 for circ in death_eligible_circumstances }
    maps["MULT_VICTIMS"] = 0
    maps[FAMILY] = 0
    return maps

def check_circumstance(row, counts):    
    for column in zip(age_columns, circumstance_columns, fam_columns):
        if (circumstance_help(row[column[1]]) and offender_age_help(row[column[0]])):
            counts[row[column[1]]] += 1
        elif offender_age_help(row[column[0]]) and fam_help(row[column[2]]):
            counts[FAMILY] += 1
    return counts

def fam_help(val):
    return val in fam_cases

def circumstance_help(val):
    return val in death_eligible_circumstances

def check_age(row):
    for column in age_columns:
        if (age_help(row[column])):
            return True
    return False

def age_help(val):
    return val != ' ' and (val == 'BB' or val == 'NB' or (val <= 6 and val != 0))

def check_offender_age(row):
    for off_age in offender_age_columns:
        if offender_age_help(row[off_age]):
            return True;
    return False;
    
def offender_age_help(val):
    return val == ' ' or type(val) == int and (val == 0 or val >= 18)

def count_under_18(row):
    count = 0
    for col in offender_age_columns:
        val = row[col]
        if val != ' ' and type(val) == int and (val != 0 and val < 18):
            count += 1
    return count

In [71]:
df["MAPS"] = df.apply(label_death_eligible, axis=1)

0         {2: 0, 3: 0, 5: 0, 9: 0, 46: 0, 48: 0, 49: 0, ...
1         {2: 0, 3: 0, 5: 0, 9: 0, 46: 0, 48: 0, 49: 0, ...
2         {2: 0, 3: 0, 5: 0, 9: 0, 46: 0, 48: 0, 49: 0, ...
3         {2: 0, 3: 0, 5: 0, 9: 0, 46: 0, 48: 0, 49: 0, ...
4         {2: 0, 3: 0, 5: 0, 9: 0, 46: 0, 48: 0, 49: 0, ...
5         {2: 0, 3: 0, 5: 0, 9: 0, 46: 0, 48: 0, 49: 0, ...
6         {2: 0, 3: 0, 5: 0, 9: 0, 46: 0, 48: 0, 49: 0, ...
7         {2: 0, 3: 0, 5: 0, 9: 0, 46: 0, 48: 0, 49: 0, ...
8         {2: 0, 3: 0, 5: 0, 9: 0, 46: 0, 48: 0, 49: 0, ...
9         {2: 0, 3: 0, 5: 0, 9: 0, 46: 0, 48: 0, 49: 0, ...
10        {2: 0, 3: 0, 5: 0, 9: 0, 46: 0, 48: 0, 49: 0, ...
11        {2: 0, 3: 0, 5: 0, 9: 0, 46: 0, 48: 0, 49: 0, ...
12        {2: 0, 3: 0, 5: 0, 9: 0, 46: 0, 48: 0, 49: 0, ...
13        {2: 0, 3: 0, 5: 0, 9: 0, 46: 0, 48: 0, 49: 0, ...
14        {2: 0, 3: 0, 5: 1, 9: 0, 46: 0, 48: 0, 49: 0, ...
15        {2: 0, 3: 0, 5: 0, 9: 0, 46: 0, 48: 0, 49: 0, ...
16        {2: 0, 3: 0, 5: 0, 9: 0, 46: 0

In [78]:
COUNTY = "V8"
YEAR = "V6"
death_eligible_circumstances = { 3, 5, 2, 9, 46, 49, 48 }


newDF = { 'YEAR' : [],
        'COUNTY' : [],
         'RAPE' : [],
         'ROBBERY' : [],
         'BURGLARY' : [],
         'ARSON' : [],
         'GANG' : [],
         "INSTITUTIONAL" : [],
         "SNIPER" : [],
         "MULT_VICTIMS" : [],
         "FAMILY" : []
        }

df["MAPS"][12]

for county in df[COUNTY].unique():
    for year in df[YEAR].unique():
        counts = createDeathMap()
        
        sel = df.loc[(df[YEAR] == year) & (df[COUNTY] == county)]
        
        for row in sel.itertuples(index = 1):
            maps = getattr(row, "MAPS")
            for key in maps:
                counts[key] += maps[key]
        
        
        newDF['YEAR'].append(year)
        newDF['COUNTY'].append(county)
        newDF['RAPE'].append(counts[2])
        newDF['ROBBERY'].append(counts[3])
        newDF['BURGLARY'].append(counts[5])
        newDF['ARSON'].append(counts[9])
        newDF['GANG'].append(counts[46])
        newDF["INSTITUTIONAL"].append(counts[48])
        newDF["SNIPER"].append(counts[49])
        newDF["MULT_VICTIMS"].append(counts['MULT_VICTIMS'])
        newDF["FAMILY"].append(counts[FAMILY])


In [30]:
len(df[COUNTY].unique()) * len(df[YEAR].unique()) == len(newDF['YEAR'])

True

In [13]:
df['death_eligible_count'].value_counts()

0     115789
1      15637
2       3227
3       1067
4        481
5        114
6         59
7         20
8         17
9          4
10         3
11         1
Name: death_eligible_count, dtype: int64

In [79]:
# row = df.loc[[2]]

# label_death_eligible(row)
# for key in newDF:
#     print(len(set(newDF[key])))
agg = pd.DataFrame(data=newDF)
agg.to_csv("shr_agg.csv")

In [34]:
agg.to_csv("shr_agg.csv")

In [15]:
df['V6'].value_counts()

2006    15076
2007    14977
2008    14336
2009    13858
2015    13844
2010    13300
2012    13063
2011    12887
2013    12546
2014    12532
Name: V6, dtype: int64